# Finding Weightings
In this section, we will try to find the optimal weightings with the stocks that have been chosen. We will be implementing ideas from Modern Portfolio Theory by using ideas from Ivanyuk and Berzin (2020). In the next section, we will go over the theory behind this model. Then, we will implement our ideas to create a portfolio that meets our needs.

## Explanation
The main assumption behind the Markowitz Model is that investors are willing to accept more risk if they get more returns. In other words, the best portfolio is the one that provides the most returns for the same amount of risk. 

Firstly, we decided to plot out a portfolio that assigns random weightings to random stocks. We plot the returns on a scatter graph.

Note that the following code is only an example and is not coded to be dynamic. We include an implementation in the section after.

# References
Ivanyuk, V., & Berzin, D. (2020). *An Algorithm for Constructing an Efficient Investment Portfolio*. In Advances in Intelligent 

&ensp; Systems and Computing (pp. 482–490). Springer International Publishing. https://doi.org/10.1007/978-3-030-63322-6_39
